In [10]:
import pandas as pd
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from textblob import TextBlob
from nltk.corpus import stopwords
from collections import Counter
import warnings; warnings.simplefilter('ignore')
import nltk
import re
import string
from string import punctuation
from nltk import ngrams
from nltk.tokenize import word_tokenize 
from nltk.stem import SnowballStemmer

In [2]:
df1 = pd.read_csv("df_acne.csv")
df2 = pd.read_csv("df_ADHD.csv")
df3 = pd.read_csv("df_HIV.csv")
df4 = pd.read_csv("df_allergies.csv")
df5 = pd.read_csv("df_angina.csv")
df6 = pd.read_csv("df_anxiety.csv")
df7 = pd.read_csv("df_asthma.csv")
df8 = pd.read_csv("df_bipolar.csv")
df9 = pd.read_csv("df_bronchitis.csv")
df10 = pd.read_csv("df_cancer.csv")
df11 = pd.read_csv("df_cholesterol.csv")
df12 = pd.read_csv("df_cold.csv")
df13 = pd.read_csv("df_constipation.csv")
df14 = pd.read_csv("df_depression.csv")
df15 = pd.read_csv("df_diabetes1.csv")
df16 = pd.read_csv("df_diabetes2.csv")
df17 = pd.read_csv("df_eczema.csv")
df18 = pd.read_csv("df_hayfever.csv")
df19 = pd.read_csv("df_herpes.csv")
df20 = pd.read_csv("df_hypertension.csv")
df21 = pd.read_csv("df_ibs.csv")
df22 = pd.read_csv("df_insomnia.csv")
df23 = pd.read_csv("df_migraine.csv")
df24 = pd.read_csv("df_pain.csv")
df25 = pd.read_csv("df_pneumonia.csv")
df26 = pd.read_csv("df_schitzophrenia.csv")
df27 = pd.read_csv("df_seizures.csv")
df28 = pd.read_csv("df_chlamydia.csv")
df29 = pd.read_csv("df_obesity.csv")

frames = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19,df20,df21,df22,df23,df24,df25,df26,df27,df28,df29]
df = pd.concat(frames)
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True, drop=True)

In [3]:
df = df[pd.notnull(df['Rating'])]
df.drop(['Taken'],axis=1,inplace=True )
df['Date_split'] = df['Date'].str.split(' ')
df['Month'] = df['Date_split'].str[0].astype('str')
df['Year'] = df['Date_split'].str[2].astype('int')
df.drop(['Date', 'Date_split'], axis=1, inplace=True)

In [4]:
def remove_jargon(k):
    return re.sub('\t\t\t', '', k)

df['Review'] = df['Review'].apply(remove_jargon)

def lower_case(k):
    return k.lower()

df['Review'] = df['Review'].apply(lower_case)
df['Drug'] = df['Drug'].apply(lower_case)

df.reset_index(inplace=True, drop=True)

def count_words(k):
    return len(k)

df['Review_length'] = df['Review'].apply(count_words)

In [8]:
df.drop(['Major Interaction'], axis=1, inplace = True)
df.drop(['Moderate Interaction'], axis=1, inplace=True)
df.drop(['Minor Interaction'], axis=1, inplace=True)
df.drop(['Dosage'], axis=1, inplace=True)

In [15]:
stops = set(stopwords.words('english'))
punctuation = punctuation + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'

In [13]:
contractions_dict = {
    "ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because",
    "could've": "could have", "couldn't": "could not", "couldn't've": "could not have", "didn't": "did not", "doesn't": "does not",
    "doesn’t": "does not", "don't": "do not", "don’t": "do not", "hadn't": "had not", "hadn't've": "had not have", "hasn't": "has not",
    "haven't": "have not", "he'd": "he had", "he'd've": "he would have", "he'll": "he will", "he'll've": "he will have", "he's": "he is",
    "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "i'd": "i would", "i'd've": "i would have",
    "i'll": "i will", "i'll've": "i will have", "i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have",
    "it'll": "it will", "it'll've": "it will have", "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not","might've": "might have",
    "mightn't": "might not", "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have",
    "needn't": "need not", "needn't've": "need not have", "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have",
    "shan't": "shall not","sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have",
    "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not",
    "shouldn't've": "should not have", "so've": "so have", "so's": "so is", "that'd": "that would", "that'd've": "that would have",
    "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "they'd": "they would",
    "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have",
    "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
    "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
    "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
    "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is",
    "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have",
    "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y’all": "you all", "y'all'd": "you all would",
    "y'all'd've": "you all would have", "y'all're": "you all are", "y'all've": "you all have", "you'd": "you would", "you'd've": "you would have",
    "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", "ain’t": "am not", "aren’t": "are not",
    "can’t": "cannot", "can’t’ve": "cannot have", "’cause": "because", "could’ve": "could have", "couldn’t": "could not", "couldn’t’ve": "could not have",
    "didn’t": "did not", "doesn’t": "does not", "don’t": "do not", "don’t": "do not", "hadn’t": "had not", "hadn’t’ve": "had not have",
    "hasn’t": "has not", "haven’t": "have not", "he’d": "he had", "he’d’ve": "he would have", "he’ll": "he will", "he’ll’ve": "he will have",
    "he’s": "he is", "how’d": "how did", "how’d’y": "how do you", "how’ll": "how will", "how’s": "how is", "i’d": "i would", "i’d’ve": "i would have",
    "i’ll": "i will", "i’ll’ve": "i will have", "i’m": "i am", "i’ve": "i have", "isn’t": "is not", "it’d": "it would", "it’d’ve": "it would have",
    "it’ll": "it will", "it’ll’ve": "it will have", "it’s": "it is", "let’s": "let us", "ma’am": "madam", "mayn’t": "may not",
    "might’ve": "might have", "mightn’t": "might not", "mightn’t’ve": "might not have", "must’ve": "must have", "mustn’t": "must not",
    "mustn’t’ve": "must not have", "needn’t": "need not", "needn’t’ve": "need not have", "o’clock": "of the clock",
    "oughtn’t": "ought not", "oughtn’t’ve": "ought not have", "shan’t": "shall not", "sha’n’t": "shall not", "shan’t’ve": "shall not have",
    "she’d": "she would", "she’d’ve": "she would have", "she’ll": "she will", "she’ll’ve": "she will have", "she’s": "she is",
    "should’ve": "should have", "shouldn’t": "should not", "shouldn’t’ve": "should not have", "so’ve": "so have", "so’s": "so is",
    "that’d": "that would", "that’d’ve": "that would have", "that’s": "that is", "there’d": "there would", "there’d’ve": "there would have",
    "there’s": "there is", "they’d": "they would", "they’d’ve": "they would have", "they’ll": "they will", "they’ll’ve": "they will have",
    "they’re": "they are", "they’ve": "they have", "to’ve": "to have", "wasn’t": "was not", "we’d": "we would", "we’d’ve": "we would have",
    "we’ll": "we will", "we’ll’ve": "we will have", "we’re": "we are", "we’ve": "we have", "weren’t": "were not", "what’ll": "what will",
    "what’ll’ve": "what will have", "what’re": "what are", "what’s": "what is", "what’ve": "what have", "when’s": "when is",
    "when’ve": "when have", "where’d": "where did", "where’s": "where is", "where’ve": "where have", "who’ll": "who will",
    "who’ll’ve": "who will have", "who’s": "who is", "who’ve": "who have","why’s": "why is", "why’ve": "why have", "will’ve": "will have",
    "won’t": "will not", "won’t’ve": "will not have", "would’ve": "would have", "wouldn’t": "would not", "wouldn’t’ve": "would not have",
    "y’all": "you all", "y’all": "you all", "y’all’d": "you all would", "y’all’d’ve": "you all would have", "y’all’re": "you all are",
    "y’all’ve": "you all have", "you’d": "you would", "you’d’ve": "you would have", "you’ll": "you will", "you’ll’ve": "you will have",
    "you’re": "you are", "you’re": "you are", "you’ve": "you have"
}
contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))

def expand_contractions(s, contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)

In [16]:
df['review_clean'] = df['Review'].apply(lambda x: expand_contractions(x))
df['review_clean'] = df['review_clean'].map(lambda x: ' '.join(word for word in word_tokenize(x) if word not in stops))
df['review_clean'] = df['review_clean'].map(lambda x: ' '.join(word for word in word_tokenize(x) if word not in punctuation))

In [17]:
df['Rating_Binary'] = df['Rating'].apply(lambda x: 1 if x > 5.0 else 0)

In [18]:
df.head()

,Rating,Helpful,Review,Drug,Condition,Month,Year,Review_length,review_clean,Rating_Binary
0,1.0,19,accutane (isotretinoin):“i took this poison fo...,isotretinoin,Acne,September,2020,566,accutane isotretinoin took poison 6 months com...,0
1,10.0,6,myorisan (isotretinoin):“this is my sixth and ...,isotretinoin,Acne,September,2020,927,myorisan isotretinoin sixth last month taking ...,1
2,2.0,9,accutane (isotretinoin):“there is no doubt thi...,isotretinoin,Acne,August,2020,725,accutane isotretinoin doubt drug accutane work...,0
3,10.0,9,claravis (isotretinoin):“a month after finishi...,isotretinoin,Acne,August,2020,513,claravis isotretinoin month finishing claravis...,1
4,3.0,9,accutane (isotretinoin):“i took this horrible ...,isotretinoin,Acne,August,2020,195,accutane isotretinoin took horrible drug 35 ye...,0


In [19]:
df.to_csv("df.csv")